In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ajuste import ajustar_precos

In [2]:
# Histórico dos preços de fechamento do HGLG11
hglg11_precos = pd.read_csv('data/hglg11_precos.csv', parse_dates=['data'])
hglg11_precos

,data,preco
0,2011-03-02,104.40
1,2011-03-21,104.40
2,2011-03-28,104.40
3,2011-03-31,104.40
4,2011-04-05,104.40
...,...,...
2767,2022-12-05,162.84
2768,2022-12-06,162.95
2769,2022-12-07,162.18
2770,2022-12-08,162.20


In [3]:
# Histórico de dividendos do HGLG11 em que a data base foi ajustada
# para ser necessariamente em dia com pregão
hglg11_divs = (
    pd.read_csv('data/hglg11_dividendos.csv', parse_dates=['data_com'])
    # Remover as colunas que não serão utilizadas
    .drop(columns=['tipo', 'data_pagamento'])
)
hglg11_divs

,data_com,valor_div
0,2011-03-31,0.72
1,2011-04-29,0.75
2,2011-05-31,0.79
3,2011-06-30,0.80
4,2011-07-29,0.81
...,...,...
136,2022-07-29,1.10
137,2022-08-31,1.10
138,2022-09-30,1.10
139,2022-10-31,1.10


In [4]:
hglg11_precos = ajustar_precos(hglg11_precos, hglg11_divs)
hglg11_precos

,data,preco,preco_aj,valor_div
0,2011-03-02,104.40,37.450552,NaN
1,2011-03-21,104.40,37.450552,NaN
2,2011-03-28,104.40,37.450552,NaN
3,2011-03-31,104.40,37.450552,0.72
4,2011-04-05,104.40,37.710625,NaN
...,...,...,...,...
2767,2022-12-05,162.84,162.840000,NaN
2768,2022-12-06,162.95,162.950000,NaN
2769,2022-12-07,162.18,162.180000,NaN
2770,2022-12-08,162.20,162.200000,NaN


In [5]:
# Calcular o dividend_yield
hglg11_precos["dividend_yield(%)"] = 100 * hglg11_precos["valor_div"] / hglg11_precos["preco"]
hglg11_precos.query("valor_div > 0")

,data,preco,preco_aj,valor_div,dividend_yield(%)
3,2011-03-31,104.40,37.450552,0.72,0.689655
13,2011-04-27,104.40,37.710625,0.75,0.718391
27,2011-05-31,104.30,37.947112,0.79,0.757430
39,2011-06-30,106.30,38.969935,0.80,0.752587
48,2011-07-21,109.00,40.262777,0.81,0.743119
...,...,...,...,...,...
2680,2022-07-29,168.64,163.207107,1.10,0.652277
2703,2022-08-31,172.49,168.029090,1.10,0.637718
2724,2022-09-30,171.50,168.136933,1.10,0.641399
2744,2022-10-31,165.39,163.193471,1.10,0.665095


In [6]:
hglg11_precos.query('data >= "2022-11-27"')

,data,preco,preco_aj,valor_div,dividend_yield(%)
2762,2022-11-28,162.90,161.812748,NaN,NaN
2763,2022-11-29,163.50,162.408743,NaN,NaN
2764,2022-11-30,164.81,163.710000,1.1,0.667435
2765,2022-12-01,162.71,162.710000,NaN,NaN
2766,2022-12-02,162.80,162.800000,NaN,NaN
2767,2022-12-05,162.84,162.840000,NaN,NaN
2768,2022-12-06,162.95,162.950000,NaN,NaN
2769,2022-12-07,162.18,162.180000,NaN,NaN
2770,2022-12-08,162.20,162.200000,NaN,NaN
2771,2022-12-09,162.97,162.970000,NaN,NaN


In [7]:
# Comparar os preços com e sem ajuste do HGLG11 com o Plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco"],
        mode="lines",
        line=dict(color="#97c9cc"),
        name="Preço sem ajuste",        
    )
)
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco_aj"],
        mode="lines",
        line=dict(color="#02878e"),
        name="Preço com ajuste",
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento do HGLG11 <br>(com e sem ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()
# py.plot(fig, filename = 'hglg_petr_1', auto_open=True)

In [8]:
# Valorização do HGLG11 com e sem ajuste de dividendos
valorizacao_sa = (hglg11_precos["preco"].iloc[-1] / hglg11_precos["preco"].iloc[0] - 1) * 100
valorizacao_ca = (hglg11_precos["preco_aj"].iloc[-1] / hglg11_precos["preco_aj"].iloc[0] - 1) * 100
print(f"Valorização do HGLG11 com ajuste de dividendos: {valorizacao_sa:.2f}%")
print(f"Valorização do HGLG11 sem ajuste de dividendos: {valorizacao_ca:.2f}%")

Valorização do HGLG11 com ajuste de dividendos: 56.10%
Valorização do HGLG11 sem ajuste de dividendos: 335.16%


In [9]:
# Carregar os dados de preços e dividendos do PETR4
petr4_precos = (
    pd.read_csv('data/petr4_precos.csv', parse_dates=['data'])
    # Concidir com o período de negociação do HGLG11
    .query('data >= "2011-03-02"')
    # Renomear com o preço desejado para "preco"
    .rename(columns={'preult': 'preco'})
    # Selecionar apenas as colunas de interesse
    [['data', 'preco']]
)
petr4_precos

,data,preco
2842,2011-03-02,28.900000
2843,2011-03-03,28.959999
2844,2011-03-04,29.080000
2845,2011-03-09,29.080000
2846,2011-03-10,28.120001
...,...,...
5762,2022-12-07,25.350000
5763,2022-12-08,24.780001
5764,2022-12-09,24.709999
5765,2022-12-12,23.910000


In [10]:
petr4_divs = pd.read_csv('data/petr4_dividendos.csv', parse_dates=['data_com'])
dividendos_12m = petr4_divs.query('data_com >= "2021-12-15"')["valor_div"].sum()
print(f"Dividendos pagos nos últimos 12 meses: R$ {dividendos_12m:.2f}")
petr4_divs.tail()

Dividendos pagos nos últimos 12 meses: R$ 16.77


,data_com,valor_div
27,2021-12-01,3.250487
28,2022-04-13,2.970249
29,2022-05-23,3.715490
30,2022-08-11,6.732003
31,2022-11-21,3.348900


In [11]:
# Ajustart os preços de fechamento da PETR4
petr4_precos = ajustar_precos(petr4_precos, petr4_divs)
petr4_precos

,data,preco,preco_aj,valor_div
0,2011-03-02,28.900000,10.795776,NaN
1,2011-03-03,28.959999,10.818189,NaN
2,2011-03-04,29.080000,10.863016,NaN
3,2011-03-09,29.080000,10.863016,NaN
4,2011-03-10,28.120001,10.504402,NaN
...,...,...,...,...
2920,2022-12-07,25.350000,25.350000,NaN
2921,2022-12-08,24.780001,24.780001,NaN
2922,2022-12-09,24.709999,24.709999,NaN
2923,2022-12-12,23.910000,23.910000,NaN


In [12]:
# Calcular o dividend_yield
petr4_precos["dividend_yield(%)"] = 100 * petr4_precos["valor_div"] / petr4_precos["preco"]
petr4_precos.query("valor_div > 0")

,data,preco,preco_aj,valor_div,dividend_yield(%)
11,2011-03-21,28.129999,10.508137,0.152720,0.542908
37,2011-04-28,25.450001,9.558903,0.124200,0.488016
46,2011-05-11,24.600000,9.284959,0.200000,0.813008
104,2011-08-02,23.500000,8.942481,0.200000,0.851064
174,2011-11-11,21.959999,8.428192,0.200000,0.910747
208,2012-01-02,21.730000,8.416573,0.200000,0.920387
259,2012-03-19,24.350000,9.518975,0.122670,0.503778
296,2012-05-11,19.530001,7.673384,0.200000,1.024065
532,2013-04-29,20.420000,8.106078,0.782970,3.834329
763,2014-04-02,15.560000,6.423098,0.997570,6.411118


In [13]:
# Plotar os preços da PETR4 e HGLG11 sem ajustes
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=petr4_precos["data"],
        y=petr4_precos["preco"],
        mode="lines",
        name="PETR4",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco"],
        mode="lines",
        name="HGLG11",
        line=dict(color="#02878e"),
    )
)
# Adicionar os pontos de dividendos da PETR4 no próprio gráfico de preços
fig.add_trace(
    go.Scatter(
        x=petr4_precos.query('valor_div > 0').data,
        y=petr4_precos.query('valor_div > 0')["preco"],
        mode="markers",
        name="Dividendos da PETR4",
        marker=dict(
            color="black",
            size=(20 * petr4_precos.query('valor_div > 0')["dividend_yield(%)"]) ** 0.5,
            symbol="circle"),
    )
)
# Adicionar os pontos de dividendos da HGLG11 no próprio gráfico de preços, sendo que o tamanho dos pontos é proporcional ao valor do dividendo
fig.add_trace(
    go.Scatter(
        x=hglg11_precos.query('valor_div > 0').data,
        y=hglg11_precos.query('valor_div > 0')["preco"],
        mode="markers",
        name="Dividendos do HGLG11",
        marker=dict(
            color="black",
            size=(20 * hglg11_precos.query('valor_div > 0')["dividend_yield(%)"]) ** 0.5,
            symbol="circle"),
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento <br> (sem ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()
# py.plot(fig, filename = 'hglg_petr_2', auto_open=True)

In [14]:
# Adicionar uma coluna com o preço normalizado em petr4_precos e hglg11_precos
petr4_precos["preco_normalizado"] = 100 * petr4_precos["preco"] / petr4_precos["preco"].iloc[0]
hglg11_precos["preco_normalizado"] = 100 * hglg11_precos["preco"] / hglg11_precos["preco"].iloc[0]

In [15]:
# Comparar os preços ajustados e normalizados (base 100) para o início da série dos dois ativos com o Plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=petr4_precos["data"],
        y=petr4_precos["preco_normalizado"],
        mode="lines",
        name="PETR4",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco_normalizado"],
        mode="lines",
        name="HGLG11",
        line=dict(color="#02878e"),
    )
)
fig.update_layout(
    # mudar a fonte
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento <br>(normalizados e sem ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()
# py.plot(fig, filename = 'hglg_petr_3', auto_open=True)

In [16]:
petr4_precos.query('valor_div > 0')

,data,preco,preco_aj,valor_div,dividend_yield(%),preco_normalizado
11,2011-03-21,28.129999,10.508137,0.152720,0.542908,97.335637
37,2011-04-28,25.450001,9.558903,0.124200,0.488016,88.062287
46,2011-05-11,24.600000,9.284959,0.200000,0.813008,85.121107
104,2011-08-02,23.500000,8.942481,0.200000,0.851064,81.314879
174,2011-11-11,21.959999,8.428192,0.200000,0.910747,75.986156
208,2012-01-02,21.730000,8.416573,0.200000,0.920387,75.190311
259,2012-03-19,24.350000,9.518975,0.122670,0.503778,84.256055
296,2012-05-11,19.530001,7.673384,0.200000,1.024065,67.577858
532,2013-04-29,20.420000,8.106078,0.782970,3.834329,70.657439
763,2014-04-02,15.560000,6.423098,0.997570,6.411118,53.840830


In [18]:
# Adicionar uma coluna com o preço ajustado e normalizado em petr4_precos e hglg11_precos
petr4_precos["preco_aj_normalizado"] = 100 * petr4_precos["preco_aj"] / petr4_precos["preco_aj"].iloc[0]
hglg11_precos["preco_aj_normalizado"] = 100 * hglg11_precos["preco_aj"] / hglg11_precos["preco_aj"].iloc[0]

In [19]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=petr4_precos["data"],
        y=petr4_precos["preco_aj_normalizado"],
        mode="lines",
        name="PETR4",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco_aj_normalizado"],
        mode="lines",
        name="HGLG11",
        line=dict(color="#02878e"),
        
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento <br> (normalizados e com ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()